In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii",
                  parse_dates = [[0,1]], infer_datetime_format= True, header = None)


In [10]:
data.columns = ['Date', 'Value']
data.head()

,Date,Value
0,1950-01-01,-0.060310
1,1950-02-01,0.626810
2,1950-03-01,-0.008128
3,1950-04-01,0.555100
4,1950-05-01,0.071577


In [12]:
data.index = data.Date
data.head()

,Date,Value
Date,,
1950-01-01,1950-01-01,-0.060310
1950-02-01,1950-02-01,0.626810
1950-03-01,1950-03-01,-0.008128
1950-04-01,1950-04-01,0.555100
1950-05-01,1950-05-01,0.071577


In [15]:
data = data.drop('Date', 1)
data.head()

,Value
Date,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [20]:
data['1950':'1951']

,Value
Date,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577
1950-06-01,0.538570
1950-07-01,-0.802480
1950-08-01,-0.851010
1950-09-01,0.357970


In [21]:
# What kind of index do we have?
type(data.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [31]:
data['1951-11-11':'1951-01-12']

,Value
Date,


In [30]:
data['1951-11-11':'1951-12-01']

,Value
Date,
1951-12-01,1.9872


In [25]:
# What if we want a period index?
data_pd = data.to_period()

In [26]:
data_pd['1951-11-11':'1951-11-12']

,Value
Date,
1951-11,-0.068519


In [27]:
data_pd['1951-11-11':'1952-01-12']

,Value
Date,
1951-11,-0.068519
1951-12,1.987200
1952-01,0.368250


In [32]:
# How do various data loads perform?
import timeit

print("infer_datetime_format = True, no date parser")
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = True, header = None,)

print("infer_datetime_format = False, no date parser")
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = False, header = None,)

print("infer_datetime_format = True, date parser provided")
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = True, date_parser = dateparse,  header = None,)

print("infer_datetime_format = False, date parser provided")
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')
%timeit pd.read_fwf("http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii", parse_dates = [[0, 1]], infer_datetime_format = False, date_parser = dateparse,  header = None,)

infer_datetime_format = True, no date parser
62.1 ms ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
infer_datetime_format = False, no date parser
57.4 ms ± 975 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
infer_datetime_format = True, date parser provided
70.1 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
infer_datetime_format = False, date parser provided
73.1 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
# What if you already have the data frame and want to parse columns?
df = pd.DataFrame({'year': [2015, 2016],'month': [2, 3],'day': [4, 5],'hour': [2, 3]})
df
# Note: the column name must be as specified

,day,hour,month,year
0,4,2,2,2015
1,5,3,3,2016


In [34]:
pd.to_datetime(df)

0   2015-02-04 02:00:00
1   2016-03-05 03:00:00
dtype: datetime64[ns]

In [35]:
pd.to_datetime(df[['year', 'month', 'day']])


0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

In [36]:
# Let's experiment with truncate convenience function
ts = pd.Series(range(10), index = pd.date_range('7/31/2015', freq = 'M', periods = 10))
ts

2015-07-31    0
2015-08-31    1
2015-09-30    2
2015-10-31    3
2015-11-30    4
2015-12-31    5
2016-01-31    6
2016-02-29    7
2016-03-31    8
2016-04-30    9
Freq: M, dtype: int64

In [38]:
ts.truncate(before='10/31/2015', after='12/31/2015')
# Note: the data is truncated, without breaking the frequency 

2015-10-31    3
2015-11-30    4
2015-12-31    5
Freq: M, dtype: int64

In [39]:
# You can truncate in a way that breaks frequency
ts[[0, 2, 6]].index

DatetimeIndex(['2015-07-31', '2015-09-30', '2016-01-31'], dtype='datetime64[ns]', freq=None)

In [40]:
# It will save you when it can
ts.ix[0:10:2].index

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


DatetimeIndex(['2015-07-31', '2015-09-30', '2015-11-30', '2016-01-31',
               '2016-03-31'],
              dtype='datetime64[ns]', freq='2M')